<a href="https://colab.research.google.com/github/bkkaggle/pytorch-CycleGAN-and-pix2pix/blob/master/pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
import os

# check if `pytorch-CycleGAN-and-pix2pix` is already cloned
if not os.path.exists('pytorch-CycleGAN-and-pix2pix'):
	!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pwd

In [ ]:
%pip install -q -r requirements.txt

# Datasets

put the dataset in the `datasets` folder

Each dataset should have the following directory structure:

```
datasets
└── training_dataset
	├── train
	├── test
```

# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

- Change the `--dataroot` and `--name` to your own dataset's path and model's name. 

- Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size.

In [ ]:
# the below code train the full dataset in one stage (in which the model is trained on both the road and river images)
# use this line as baseline
! python train.py --dataroot ./datasets/training_dataset/ --name ROAD_RIVER_pix2pix --model pix2pix --direction AtoB --display_id -1

# use nohup to run the training in the background
# ! nohup python train.py ... &

# Train 2 domain-specific models
- one for RIVER
- one for ROAD

Add argument:
- `--n_epochs=400 (default is 200)`


In [ ]:
# ! python train.py --dataroot ./datasets/ROAD --name ROAD --model pix2pix --direction AtoB --display_id 0
# ! python train.py --dataroot ./datasets/RIVER --name RIVER --model pix2pix --direction AtoB --display_id 0

# use nohup to run the training in the background
# ! nohup python train.py ... road.out &	# for ROAD
# ! nohup python train.py ... river.out &	# for RIVER

# Recording use:
# [1] 7906
# [2] 8494

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [ ]:
# make sure the pre-trained weight is in the checkpoints folder
!ls checkpoints/

In [ ]:
# public test with whole dataset --num_test 1000
! python test.py --dataroot ./datasets/public_testing_dataset_1 --name ROAD_RIVER_pix2pix --model pix2pix --direction AtoB --num_test 1000

In [ ]:
print(f"Size: {len(os.listdir('./results/ROAD_RIVER_pix2pix/test_latest/images/'))}")

# Transform the results into AI CUP format

The results are stored in `./ROAD_pix2pix/test_latest/images/`.

And there are 2 types of results: `real_A` and `fake_B`.

The `real_A` is the original image, and the `fake_B` is the transformed image.

Store the `{Prefix}_fake_B.png` as `{Prefix}.jpg` to `./ROAD_pix2pix/test_latest/submission/`.

In [ ]:
# store the fake images to the `./results/ROAD_pix2pix/test_latest/submission/` folder

import shutil

source_folder = './results/ROAD_RIVER_pix2pix/test_latest/images'
target_folder = './results/ROAD_RIVER_pix2pix/test_latest/submission'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

for image_name in os.listdir(source_folder):
    if 'fake' in image_name:
        shutil.copy(os.path.join(source_folder, image_name), os.path.join(target_folder, image_name))
print(f"Size: {len(os.listdir(target_folder))}")

In [ ]:
# rename the fake images
for image_name in os.listdir(target_folder):
    # ignore the postfix: _fake_B.png
    # and transform the image from png to jpg
    new_name = image_name.replace('_fake_B.png', '.jpg')
    os.rename(os.path.join(target_folder, image_name), os.path.join(target_folder, new_name))
print(f"Size: {len(os.listdir(target_folder))}")

In [ ]:
# zip the fake images
shutil.make_archive(target_folder, 'zip', target_folder)